In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.0 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 3.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=f28b8d2865e373035f02c8b10498877c15e2a6bf8146d58e7a4e12897287f26c
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [11]:

from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .config("spark.driver.host","172.27.1.12") \
      .config('spark.driver.port','8080') \
      .config("hive.metastore.uris", "thrift://172.27.1.8:9083")\
      .config("spark.sql.warehouse.dir", "/users/hive/warehouse")\
      .enableHiveSupport() \
      .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 19:52:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sql("select * from retail_db.categories").show(truncate=False)

+-----------+----------------------+-------------------+
|category_id|category_department_id|category_name      |
+-----------+----------------------+-------------------+
|1          |2                     |Football           |
|2          |2                     |Soccer             |
|3          |2                     |Baseball & Softball|
|4          |2                     |Basketball         |
|5          |2                     |Lacrosse           |
|6          |2                     |Tennis & Racquet   |
|7          |2                     |Hockey             |
|8          |2                     |More Sports        |
|9          |3                     |Cardio Equipment   |
|10         |3                     |Strength Training  |
|11         |3                     |Fitness Accessories|
|12         |3                     |Boxing & MMA       |
|13         |3                     |Electronics        |
|14         |3                     |Yoga & Pilates     |
|15         |3                 

In [29]:
spark.sql("select * from retail_db.customers").show(truncate=False)

+-----------+--------------+--------------+--------------+-----------------+---------------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street            |customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+---------------------------+-------------+--------------+----------------+
|1          |Richard       |Hernandez     |XXXXXXXXX     |XXXXXXXXX        |6303 Heather Plaza         |Brownsville  |TX            |78521           |
|2          |Mary          |Barrett       |XXXXXXXXX     |XXXXXXXXX        |9526 Noble Embers Ridge    |Littleton    |CO            |80126           |
|3          |Ann           |Smith         |XXXXXXXXX     |XXXXXXXXX        |3422 Blue Pioneer Bend     |Caguas       |PR            |00725           |
|4          |Mary          |Jones         |XXXXXXXXX     |XXXXXXXXX        |8324 Little Common

In [30]:
spark.sql("select * from retail_db.departments").show(truncate=False)

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|2            |Fitness        |
|3            |Footwear       |
|4            |Apparel        |
|5            |Golf           |
|6            |Outdoors       |
|7            |Fan Shop       |
+-------------+---------------+



In [31]:
spark.sql("select * from retail_db.orders").show(truncate=False)

+--------+---------------------+-----------------+---------------+
|order_id|order_date           |order_customer_id|order_status   |
+--------+---------------------+-----------------+---------------+
|1       |2013-07-25 00:00:00.0|11599            |CLOSED         |
|2       |2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|12111            |COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827             |CLOSED         |
|5       |2013-07-25 00:00:00.0|11318            |COMPLETE       |
|6       |2013-07-25 00:00:00.0|7130             |COMPLETE       |
|7       |2013-07-25 00:00:00.0|4530             |COMPLETE       |
|8       |2013-07-25 00:00:00.0|2911             |PROCESSING     |
|9       |2013-07-25 00:00:00.0|5657             |PENDING_PAYMENT|
|10      |2013-07-25 00:00:00.0|5648             |PENDING_PAYMENT|
|11      |2013-07-25 00:00:00.0|918              |PAYMENT_REVIEW |
|12      |2013-07-25 00:00:00.0|1837             |CLOSED      

In [32]:
spark.sql("select * from retail_db.order_items").show(truncate=False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|1            |1                  |957                  |1                  |299.98             |299.98                  |
|2            |2                  |1073                 |1                  |199.99             |199.99                  |
|3            |2                  |502                  |5                  |250.0              |50.0                    |
|4            |2                  |403                  |1                  |129.99             |129.99                  |
|5            |4                  |897                  |2                  |49.98              |24.99                   |
|6            |4

In [33]:
spark.sql("select * from retail_db.products").show(truncate=False)

+----------+-------------------+---------------------------------------------+-------------------+-------------+------------------------------------------------------------------------------------------------+
|product_id|product_category_id|product_name                                 |product_description|product_price|product_image                                                                                   |
+----------+-------------------+---------------------------------------------+-------------------+-------------+------------------------------------------------------------------------------------------------+
|1         |2                  |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|                   |59.98        |http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy           |
|2         |2                  |Under Armour Men's Highlight MC Football Clea|                   |129.99       |http://images.acmesports.sports/Under+Armour+Men

### Enunciado 1
Mostrar un Top 20 de clientes que mas productos compraron con sus respectivos montos.

In [34]:
spark.sql("""SELECT
    customer_id, customer_fname, customer_lname, customer_email, sum(order_item_quantity) as quantity_item_total, sum(order_item_subtotal)as total
FROM
    retail_db.customers as c
INNER JOIN
    retail_db.orders as o
    ON c.customer_id = o.order_customer_id
INNER JOIN
    retail_db.order_items as oi
    ON o.order_id = oi.order_item_order_id
WHERE order_status <> 'CANCELED'
GROUP BY customer_id, customer_fname, customer_lname, customer_email
ORDER BY  total DESC
LIMIT 20""").show()

+-----------+--------------+--------------+--------------+-------------------+------------------+
|customer_id|customer_fname|customer_lname|customer_email|quantity_item_total|             total|
+-----------+--------------+--------------+--------------+-------------------+------------------+
|        791|          Mary|         Smith|     XXXXXXXXX|                 82|10524.170177459717|
|       8766|          Mary|        Duncan|     XXXXXXXXX|                 87| 9296.140186309814|
|       1657|         Betty|      Phillips|     XXXXXXXXX|                111| 9223.710151672363|
|       2641|         Betty|        Spears|     XXXXXXXXX|                 90| 9130.920223236084|
|       1288|        Evelyn|      Thompson|     XXXXXXXXX|                 89|  9019.11019897461|
|       3710|        Ashley|         Smith|     XXXXXXXXX|                 89|  9019.10020828247|
|       5654|         Jerry|         Smith|     XXXXXXXXX|                103| 8904.950210571289|
|       5624|       

### Enunciado 2
Mostrar las categorías con el total de productos vendidos y los montos totales por categoría

In [36]:
spark.sql("""SELECT
    ca.category_name, sum(order_item_quantity) as item_quantity, cast(sum(order_item_subtotal) AS INT )as total
FROM retail_db.order_items as oi
INNER JOIN
    retail_db.products as p
    ON oi.order_item_product_id = p.product_id
INNER JOIN
    retail_db.categories as ca
    ON p.product_category_id = ca.category_id
GROUP BY ca.category_name;
""").show()

23/03/15 20:51:03 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
+-------------------+-------------+-------+
|      category_name|item_quantity|  total|
+-------------------+-------------+-------+
|   Camping & Hiking|        13729|4118425|
|Fitness Accessories|          856|  35601|
|         Golf Shoes|         1444| 107998|
|     Men's Footwear|        22246|2891757|
|        Electronics|         9436| 371034|
|    Women's Apparel|        62956|3147800|
|     Girls' Apparel|         3615| 151706|
|       Boxing & MMA|         1265|  85205|
|Baseball & Softball|         1785|  94057|
| Hunting & Shooting|         1239|  56848|
|       Golf Apparel|         1025|  34969|
|           Trade-In|         2822|  68721|
|      Shop By Sport|        32726|1309522|
|            Fishing|        17325|6929653|
|       Water Sports|        15540|3113844|
|        Accessories|         5349| 133671|
|        Golf Gloves|         2961| 116695|
|             

### Enunciado 3
Mostrar la categoría más vendida por ciudad

In [37]:
spark.sql("""
SELECT
    customer_city, category_name
FROM (SELECT
    customer_city, category_name, count(category_name) as quantity, DENSE_RANK () OVER ( 
                PARTITION BY customer_city 
                ORDER BY count(category_name) DESC
            ) rank
    FROM
        retail_db.customers as c
    INNER JOIN
        retail_db.orders as o
        ON c.customer_id = o.order_customer_id
    INNER JOIN
        retail_db.order_items as oi
        ON o.order_id = oi.order_item_order_id
    INNER JOIN
        retail_db.products as p
        ON oi.order_item_product_id = p.product_id
    INNER JOIN
        retail_db.categories as ca
        ON p.product_category_id = ca.category_id
    GROUP BY customer_city, category_name
    ) t
WHERE rank = 1;""").show()

23/03/15 20:51:44 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
+-----------------+--------------------+
|    customer_city|       category_name|
+-----------------+--------------------+
|        Aguadilla|              Cleats|
|          Alameda|              Cleats|
|           Albany|     Women's Apparel|
|      Albuquerque|              Cleats|
|        Algonquin|              Cleats|
|         Alhambra|     Women's Apparel|
|        Allentown|              Cleats|
|       Alpharetta|      Men's Footwear|
|         Amarillo|              Cleats|
|         Amarillo|Indoor/Outdoor Games|
|          Anaheim|     Women's Apparel|
|        Ann Arbor|     Women's Apparel|
|        Annandale|      Men's Footwear|
|        Annapolis|      Men's Footwear|
|          Antioch|Indoor/Outdoor Games|
|             Apex|              Cleats|
|           Apopka|             Fishing|
|          Arecibo|     Women's Apparel|
|        Arlington|     Women's A

### Enunciado 4
Mostrar los 5 productos más vendidos por cada ciudad y el monto recaudado

In [38]:
spark.sql("""
SELECT
    customer_city, product_name, quantity, total
FROM (SELECT
    customer_city, product_name,sum(order_item_quantity) as quantity,sum(order_item_subtotal) as total, DENSE_RANK () OVER ( 
                PARTITION BY customer_city 
                ORDER BY sum(order_item_quantity) DESC
            ) rank
    FROM
        retail_db.customers as c
    INNER JOIN
        retail_db.orders as o
        ON c.customer_id = o.order_customer_id
    INNER JOIN
        retail_db.order_items as oi
        ON o.order_id = oi.order_item_order_id
    INNER JOIN
        retail_db.products as p
        ON oi.order_item_product_id = p.product_id
    GROUP BY customer_city, product_name
    ) t
WHERE rank < 6
ORDER BY quantity DESC;
""").show()

23/03/15 20:52:20 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.


+-------------+--------------------+--------+------------------+
|customer_city|        product_name|quantity|             total|
+-------------+--------------------+--------+------------------+
|       Caguas|Perfect Fitness P...|   27276|1636287.2855148315|
|       Caguas|Nike Men's Dri-FI...|   23355|         1167750.0|
|       Caguas|O'Brien Men's Neo...|   21590|1079068.1897773743|
|       Caguas|Nike Men's Free 5...|   13530|1352864.6990203857|
|       Caguas|Under Armour Girl...|   11713| 468402.8778114319|
|      Chicago|Perfect Fitness P...|    1540| 92384.60255813599|
|  Los Angeles|Perfect Fitness P...|    1423| 85365.77238464355|
|      Chicago|Nike Men's Dri-FI...|    1391|           69550.0|
|      Chicago|O'Brien Men's Neo...|    1278| 63874.43938827515|
|     Brooklyn|Perfect Fitness P...|    1251| 75047.49206924438|
|     Brooklyn|Nike Men's Dri-FI...|    1237|           61850.0|
|  Los Angeles|Nike Men's Dri-FI...|    1194|           59700.0|
|     Brooklyn|O'Brien Me